In [92]:
Z.<X> = ZZ[]
F = GF(3*5*2^27+1)

In [93]:
def mask(n_bits):
    return (1 << n_bits) - 1
def decomp(n, n_limbs, limb_bits):
    p = 0
    for i in range(n_limbs):
        p += X^i * (n & mask(limb_bits))
        n >>= limb_bits
    return p
p = decomp(12345678, n_limbs=4, limb_bits=8); print(p)
assert p(2^8) == 12345678

188*X^2 + 97*X + 78


hmm, you can either check the poly mod 2^32, or do a widening mul

In [94]:
# check the poly % 2^32
LIMBS = dict(n_limbs=4, limb_bits=8)
a, b = [randint(0, 2^32-1) for _ in range(2)]
c = (a * b) % 2^32
print(a, b, c)
f = decomp(a, **LIMBS) * decomp(b, **LIMBS) - decomp(c, **LIMBS)
print(f)
print(f(2^8))
# yes % 2^32 here
assert f(2^8) % 2^32 == 0

3095290879 2482047104 3261134720
27048*X^6 + 62866*X^5 + 46391*X^4 + 85710*X^3 + 78279*X^2 + 14465*X + 32512
7682657758998429696


this doesnt seem to work:

In [95]:
f.change_ring(F)(2^8)

1821127190

In [135]:
# c has twice as many limbs
N_LIMBS = 4
LIMB_BITS = 8
BASE = 2^LIMB_BITS
LIMBS = dict(n_limbs=N_LIMBS, limb_bits=LIMB_BITS)
a, b = [randint(0, 2^(N_LIMBS*LIMB_BITS)-1) for _ in range(2)]
c = a * b
print(a, b, c)
f = decomp(a, **LIMBS) * decomp(b, **LIMBS) - decomp(c, n_limbs=N_LIMBS*2, limb_bits=LIMB_BITS)
print(f)
print(f(2^LIMB_BITS))
# no % 2^32 here
assert f(2^LIMB_BITS) == 0

3894059826 2496237861 9720499570660272186
-134*X^7 + 34106*X^6 + 50442*X^5 + 62771*X^4 + 52345*X^3 + 34509*X^2 + 13049*X + 1792
0


this does work:

In [127]:
print(f.change_ring(F))
f.change_ring(F)(F(2^LIMB_BITS))

2013265828*X^7 + 23655*X^6 + 38991*X^5 + 45059*X^4 + 64656*X^3 + 28556*X^2 + 29657*X + 9984


0

ignore this i dont know what im trying to do:

In [98]:
# do a probabilistic check
# after committing a,b,c, challenge alpha:
alpha = F.random_element()
# verifier computes this quotient with constraints
quotient = (f(X) - f(2^8))/(X - 2^8); print(quotient)
quotient(alpha)

-3744017816*X^6 + 16080433111765628621*X^5 + 4138611844031229904897*X^4 + 1059501234597506613015472*X^3 + 271232349588386613662120970*X^2 + 69435481514457083073808164228*X + 17775483267710049216710111204365


1996620648

In [128]:
def decomp_f(*args, **kwargs):
    return decomp(*args, **kwargs).change_ring(F)

In [129]:
ff = decomp_f(a, **LIMBS) * decomp_f(b, **LIMBS) - decomp_f(c, n_limbs=N_LIMBS*2, limb_bits=LIMB_BITS)
ff

2013265828*X^7 + 23655*X^6 + 38991*X^5 + 45059*X^4 + 64656*X^3 + 28556*X^2 + 29657*X + 9984

In [130]:
ff(2^LIMB_BITS)

0

In [131]:
f.parent()

Univariate Polynomial Ring in X over Integer Ring

In [132]:
ff.parent()

Univariate Polynomial Ring in X over Finite Field of size 2013265921

In [133]:
f.roots()

[(256, 1)]

In [134]:
ff.roots()

[(928398009, 1), (256, 1)]

In [112]:
F(4294967296)

268435454

In [114]:
F(f(268435454))

0

In [115]:
F(f(557631116))

0

In [116]:
F(f(780709709))

0

In [138]:
ax = decomp_f(a, **LIMBS)
bx = decomp_f(b, **LIMBS)
cx = decomp_f(c, n_limbs=N_LIMBS*2, limb_bits=LIMB_BITS)
ax, bx, cx

(232*X^3 + 26*X^2 + 163*X + 50,
 148*X^3 + 201*X^2 + 145*X + 37,
 134*X^7 + 230*X^6 + 38*X^5 + 219*X^4 + 172*X^3 + 138*X^2 + 232*X + 58)

In [139]:
(ax * bx - cx)(2^LIMB_BITS)

0

In [140]:
(ax * bx).roots()

[(1623187351, 1), (824694328, 1)]

In [141]:
cx[:4]

172*X^3 + 138*X^2 + 232*X + 58

In [142]:
(ax * bx)(BASE)

1690438863

In [143]:
fake_cx = 1690438863

In [144]:
(ax * bx - fake_cx)(BASE)

0

In [145]:
(ax * bx - cx).roots()

[(1089602605, 1), (256, 1)]

In [146]:
(ax * bx - cx[:4])(BASE)

808920214

In [147]:
(cx - cx[:4])(BASE)

808920214

In [148]:
(cx - cx[:4])

134*X^7 + 230*X^6 + 38*X^5 + 219*X^4